在下面的示例中，我们将研究多个进程共享和操作一个状态在本例中，四个进程将一个共享计数器递增一组次数。如果没有同步过程，计数是不正确的。如果您以一致的方式共享数据，则始终需要一种方法来同步数据的读写，否则最终会出现错误。

通常，同步方法特定于您使用的操作系统，并且通常特定于您使用的语言。这里，我们看一下使用Python库和在Python进程之间共享整数对象的基于文件的同步。

## 文件锁定

读取和写入文件将是本节中数据共享的最慢示例。

您可以在示例中看到我们的第一个功函数 9-35. 函数在本地计数器上迭代。在每次迭代中，它打开一个文件并读取现有值，将其递增一，然后在旧值上写入新值。在第一次迭代中，文件将是空的或不存在，因此它将捕获一个异常并假设值应该为零。

函数

这里给出的示例在实践中得到了简化使用上下文管理器打开文件更安全，使用with open（filename，“r”）作为f:。如果在上下文中引发异常，文件f将正确关闭。

无锁工作函数：

让我们用一个进程来运行这个示例。您可以在示例中看到输出 9-36. 工作被调用了1000次，并且正如预期的那样，它正确地计数而不丢失任何数据。在第一次读取时，它会看到一个空文件。这将引发int（）的invalid literal for int（）错误（因为int（）是在空字符串上调用的）。此错误只发生一次；之后，我们总是要读取一个有效值并将其转换为整数。

例9-36。无锁单进程基于文件计数的计时

提示
在看下面的代码之前，当两个进程同时读写同一个文件时，您希望看到哪两种类型的错误？考虑代码的两种主要状态（每个进程的开始执行和每个进程的正常运行状态）。

看看这个例子 9-37查看问题。首先，当每个进程启动时，文件为空，因此每个进程都尝试从零开始计数。第二，当一个进程写入时，另一个进程可以读取无法解析的部分写入的结果。这将导致异常，并且将回写零。这反过来又会导致我们的计数器不断重置！您能看到\n和两个值是如何被两个并发进程写入同一个打开的文件，从而导致第三个进程读取无效条目的吗？

例9-37。基于文件的无锁四进程计数计时

例子 9-38显示了使用四个进程调用work的多处理代码。请注意，我们不是使用映射，而是构建一个进程对象列表。虽然我们不使用这里的功能，但是Process对象给了我们反思每个进程状态的能力。我们鼓励您阅读文档，了解您可能希望使用流程的原因。

例9-38。run_workers 设置四个进程

例9-39。带锁和四个进程的基于文件的计数计时:

在示例中，使用紧固件会添加一行代码 9-40带@fasteners.interprocess\u锁装饰器；文件名可以是任何名称，但是使用与要锁定的文件类似的名称可能会使从命令行进行调试更容易。注意，我们不必改变内部函数；decorator在每次调用时都会获得锁，它会等到能够获得锁之后，才能继续调用工作。

例9-40。带锁的工作功能

## 锁定值

multiprocessing模块提供了几个在进程之间共享Python对象的选项。我们可以以较低的通信开销共享基本对象，也可以使用管理器共享更高级的Python对象（例如，字典和列表）（但请注意，同步成本将显著降低数据共享的速度）。

在这里，我们将使用multiprocessing.Value对象在进程之间共享一个整数。当一个值有一个锁时，锁并不能像你期望的那样防止同时读写，但不能提供原子增量。例子 9-41说明了这一点。你可以看到，我们最终得到了一个不正确的计数；这类似于我们前面看到的基于文件的不同步示例。

例9-41。没有锁定会导致错误计数

数据没有损坏，但我们确实错过了一些更新。如果您正在从一个进程写入一个值，并在其他进程中使用（但不是修改）该值，那么这种方法可能是合适的。

示例中显示了共享该值的代码 9-42. 我们必须使用value（“i”，0）指定数据类型和初始化值，我们请求一个默认值为0的有符号整数。这是作为常规参数传递给我们的Process对象的，它负责在幕后的进程之间共享相同的字节块。要访问由我们的值持有的原始对象，我们使用.Value。注意，我们要求一个就地加法，我们希望这是一个原子操作，但是这个值不支持它，所以我们的最终计数低于预期。

例9-42。无锁计数码

您可以在示例中看到正确同步的计数 9-43使用multiprocessing.Lock。

例9-43。使用锁同步对值的写入

在示例中 9-44，我们使用了一个上下文管理器（带锁）来获取锁。

例9-44。使用上下文管理器获取锁

由于锁不能提供我们所追求的粒度级别，因此它提供的基本锁会浪费一些不必要的时间。我们可以用RawValue替换该值，如示例所示 9-46，并实现增量加速。如果你有兴趣看到这个变化背后的字节码，请阅读Eli Bendersky的博客文章。

例9-46。显示更快的值和锁方法的控制台输出

如果我们共享一个原始对象数组，我们也可以使用RawArray来代替multiprocessing.Array。

我们已经研究了在一台机器上在多个进程之间分配工作的各种方法，以及在这些进程之间共享标志和同步数据共享。不过，请记住，共享数据可能会导致头痛，如果可能，请尽量避免。使机器处理所有状态共享的边缘情况是困难的；当您第一次必须调试多个进程的交互时，您将意识到为什么公认的智慧是尽可能避免这种情况。

请考虑编写运行速度稍慢但更容易被团队理解的代码。使用Redis这样的外部工具来共享状态会导致系统在运行时可以被开发人员以外的人检查，这是一种强大的方法，可以让您的团队掌握并行系统中发生的事情。

一定要记住，经过性能调整的Python代码不太可能被您的团队中更多的初级成员所理解，他们要么害怕它，要么破坏它。避免这个问题（并接受速度上的牺牲）以保持团队的高速度。

## 小结

本项目，我们研究了如何使用文件和内存锁来避免损坏数据这是一个微妙的、难以跟踪的错误源，展示了一些健壮的、轻量级的解决方案。